In [1]:
# ---------------------------------------------------- 1. Library dan module ----------------------------------------------------
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets
import os

# Variabel global untuk menyimpan titik referensi cropping dan status cropping
ref_pts = []
cropping = False
current_image_rgb = None
selected_avg_rgb = None

# Daftar parameter sesuai urutan
PARAM_LIST = [
    "Leukosit", "Nitrite", "Urobilnogen", "Protein", "pH",
    "Blood", "Specific Gravity", "Ketone", "Bilrubin", "Glucose"
]

In [3]:
# ---------------------------------------------------- 2. Fungsi pemilihan piksel RGB ----------------------------------------------------

# 1) Fungsi clik & drop
def click_and_crop(event, x, y, flags, param):
    global ref_pts, cropping, selected_avg_rgb, current_image_rgb
    image_display_for_drawing = param[0].copy() # Gambar BGR untuk digambari
    original_rgb_image_for_crop = param[1]    # Gambar RGB asli untuk di-crop

    if event == cv2.EVENT_LBUTTONDOWN:
        ref_pts = [(x, y)]
        cropping = True
    elif event == cv2.EVENT_MOUSEMOVE:
        if cropping:
            temp_image = image_display_for_drawing.copy()
            cv2.rectangle(temp_image, ref_pts[0], (x,y), (0, 255, 0), 1) # Garis tipis saat dragging
            cv2.imshow("image", temp_image)
    elif event == cv2.EVENT_LBUTTONUP:
        ref_pts.append((x, y))
        cropping = False
        cv2.rectangle(image_display_for_drawing, ref_pts[0], ref_pts[1], (0, 255, 0), 2) # Persegi final
        cv2.imshow("image", image_display_for_drawing)

        if len(ref_pts) == 2:
            x1, y1 = ref_pts[0]
            x2, y2 = ref_pts[1]
            start_x, end_x = min(x1, x2), max(x1, x2)
            start_y, end_y = min(y1, y2), max(y1, y2)

            if end_x > start_x and end_y > start_y:
                cropped_region_rgb = original_rgb_image_for_crop[start_y:end_y, start_x:end_x] # Crop dari gambar RGB asli yang disimpan di current_image_rgb
                
                avg_color_per_row = np.average(cropped_region_rgb, axis=0)
                avg_color = np.average(avg_color_per_row, axis=0)
                selected_avg_rgb = np.uint8(avg_color) # Hasil dalam kode RGB
                print(f"Rata-rata RGB terpilih: {selected_avg_rgb}")
            else:
                print("Area crop tidak valid. Silakan ulangi.")
                selected_avg_rgb = None

# 2) Fungsi memilih piksel
def select_color_from_image(image_path):
    global ref_pts, cropping, selected_avg_rgb, current_image_rgb
    
    image_bgr = cv2.imread(image_path)
    if image_bgr is None:
        print(f"Error: Tidak dapat memuat gambar di {image_path}")
        return None
    
    current_image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB) # Simpan versi RGB untuk cropping
    display_image_bgr = image_bgr.copy() # Buat salinan BGR untuk drawing dan display OpenCV

    cv2.namedWindow("image")
    cv2.setMouseCallback("image", click_and_crop, [display_image_bgr, current_image_rgb]) # Kirim display_image_bgr (untuk digambari) dan current_image_rgb (untuk di-crop)
    
    selected_avg_rgb = None 
    ref_pts = [] 

    print("Pilih area warna: Klik kiri & tahan, lalu lepas. Tekan 'c' (konfirmasi), 'r' (reset), 'q' (keluar dari pemilihan ini).")
    
    temp_display_image = display_image_bgr.copy() # Untuk reset drawing tanpa reload gambar
    while True:
        cv2.imshow("image", temp_display_image) 
        key = cv2.waitKey(1) & 0xFF

        if key == ord("r"): 
            temp_display_image = display_image_bgr.copy() # Reset tampilan ke gambar asli (BGR)
            ref_pts = []
            selected_avg_rgb = None
            cv2.setMouseCallback("image", click_and_crop, [temp_display_image, current_image_rgb]) # Reset callback dengan gambar yang bersih
            print("Seleksi direset. Silakan pilih ulang.")
        elif key == ord("c"): 
            if selected_avg_rgb is not None and len(ref_pts) == 2:
                print("Warna dikonfirmasi.")
                break
            else:
                print("Belum ada area valid yang dipilih. Silakan pilih area dan tekan 'c'.")
        elif key == ord("q"): 
            selected_avg_rgb = None 
            print("Pemilihan warna dibatalkan.")
            break
            
    cv2.destroyAllWindows()
    return selected_avg_rgb

In [83]:
# ---------------------------------------------------- 3. Fungsi pembuatan tabel referensi ----------------------------------------------------

# 1) Fungsi membuat tabel referensi
def create_reference_table_interactive():
    global selected_avg_rgb 
    
    df_reference = pd.DataFrame(columns=["No", "Par", "µr", "µg", "µb", "Hasil"])
    current_param_index = 0
    
    image_path_reference = input("Masukkan path ke gambar referensi (untuk semua parameter): ")
    image_ref_check = cv2.imread(image_path_reference)
    if image_ref_check is None:
        print(f"Error: Gambar referensi tidak ditemukan atau tidak dapat dibaca di {image_path_reference}. Pembuatan tabel referensi dibatalkan.")
        return pd.DataFrame() # Kembalikan DataFrame kosong jika gambar tidak ada

    param_entry_count = {param: 0 for param in PARAM_LIST} # Untuk nomor urut "No" per parameter

    while current_param_index < len(PARAM_LIST):
        param_name = PARAM_LIST[current_param_index]
        print(f"\n--- Menginput data untuk Parameter: {param_name} ---")

        while True: # Loop untuk satu atau lebih entri per parameter
            current_no_for_param = param_entry_count[param_name] + 1
            print(f"Data ke-{current_no_for_param} untuk {param_name}")
            
            print("Silakan pilih warna dari gambar referensi untuk parameter ini.")
            avg_rgb_values = select_color_from_image(image_path_reference)

            if avg_rgb_values is None: # Jika pemilihan dibatalkan (q) atau gagal
                action = input(f"Pemilihan warna untuk {param_name} gagal/dibatalkan. (u)langi pemilihan untuk {param_name}, (l)anjut ke parameter berikutnya, atau (s)elesai input tabel? (u/l/s): ").lower()
                if action == 'l':
                    break # Keluar dari loop entri parameter saat ini, lanjut ke parameter berikutnya
                elif action == 's':
                    print("Input tabel referensi dihentikan oleh pengguna.")
                    return df_reference # mengembalikan tabel yang sudah ada sejauh ini
                else: # 'u' atau lainnya
                    continue # Ulangi pemilihan warna untuk entri saat ini

            r_val, g_val, b_val = avg_rgb_values[0], avg_rgb_values[1], avg_rgb_values[2]

            hasil_input_valid = False
            hasil_input = ""
            while not hasil_input_valid:
                hasil_input = input(f"Masukkan hasil untuk {param_name} (e.g., +, ++, -, +/-, 5.0, atau angka): ")
                if hasil_input.strip(): # Memastikan input tidak kosong
                    hasil_input_valid = True
                else:
                    print("Input hasil tidak boleh kosong.")

            new_row_dict = {
                "No": current_no_for_param, # Nomor urut per parameter
                "Par": param_name,
                "µr": r_val,
                "µg": g_val,
                "µb": b_val,
                "Hasil": hasil_input
            }
            df_reference = pd.concat([df_reference, pd.DataFrame([new_row_dict])], ignore_index=True)
            param_entry_count[param_name] += 1
            
            print("\nData berhasil ditambahkan:")
            display(df_reference[df_reference["Par"] == param_name])
            
            next_action = input(f"Input data {param_name} selanjutnya (No. {current_no_for_param + 1})? (y/n) atau (p)indah ke parameter berikutnya, atau (e)xit & simpan tabel: ").lower()
            if next_action == 'n':
                break # Pindah ke parameter berikutnya
            elif next_action == 'p':
                break # Pindah ke parameter berikutnya
            elif next_action == 'e':
                current_param_index = len(PARAM_LIST) # Untuk selesai mengisi tabel referensi
                break 
            # Jika 'y', melanjutkan Loop untuk Parameter yang sama
        
        current_param_index += 1
            
    if not df_reference.empty:
        print("\n--- Tabel Referensi Selesai Dibuat ---")
        REFERENCE_TABLE_FILEPATH = r'C:\Users\dhana\Downloads\Tempo\pcd\FinalProjectNew\TabelReferensi2.csv'
        try:
            df_reference.to_csv(REFERENCE_TABLE_FILEPATH, index=False)
            print(f"Tabel referensi berhasil disimpan ke: {REFERENCE_TABLE_FILEPATH}")
        except Exception as e:
            print(f"Gagal menyimpan tabel referensi: {e}")
    else:
        print("Tidak ada data yang dimasukkan ke tabel referensi.")
        
    return df_reference

# 2) Panggil fungsi membuat tabel referensi
create_reference_table_interactive()

Masukkan path ke gambar referensi (untuk semua parameter):  C:\Users\dhana\Downloads\Tempo\pcd\FinalProjectNew\DipstickReferenceOriginal.jpg



--- Menginput data untuk Parameter: Leukosit ---
Data ke-1 untuk Leukosit
Silakan pilih warna dari gambar referensi untuk parameter ini.
Pilih area warna: Klik kiri & tahan, lalu lepas. Tekan 'c' (konfirmasi), 'r' (reset), 'q' (keluar dari pemilihan ini).
Rata-rata RGB terpilih: [126 120  90]
Warna dikonfirmasi.


Masukkan hasil untuk Leukosit (e.g., +, ++, -, +/-, 5.0, atau angka):  -



Data berhasil ditambahkan:


,No,Par,µr,µg,µb,Hasil
0,1,Leukosit,126,120,90,-


...

Input data Glucose selanjutnya (No. 7)? (y/n) atau (p)indah ke parameter berikutnya, atau (e)xit & simpan tabel:  e



--- Tabel Referensi Selesai Dibuat ---


,No,Par,µr,µg,µb,Hasil
0,1,Leukosit,126,120,90,-
1,2,Leukosit,124,121,102,+/-
2,3,Leukosit,100,88,81,+
3,4,Leukosit,57,34,42,++
4,5,Leukosit,39,19,37,+++
5,1,Nitrite,141,134,113,-
6,2,Nitrite,144,127,110,+
7,3,Nitrite,99,21,44,+
8,1,Urobilnogen,140,94,69,-
9,2,Urobilnogen,140,87,81,2


Tabel referensi berhasil disimpan ke: C:\Users\dhana\Downloads\Tempo\pcd\FinalProjectNew\TabelReferensi.csv


,No,Par,µr,µg,µb,Hasil
0,1,Leukosit,126,120,90,-
1,2,Leukosit,124,121,102,+/-
2,3,Leukosit,100,88,81,+
3,4,Leukosit,57,34,42,++
4,5,Leukosit,39,19,37,+++
5,1,Nitrite,141,134,113,-
6,2,Nitrite,144,127,110,+
7,3,Nitrite,99,21,44,+
8,1,Urobilnogen,140,94,69,-
9,2,Urobilnogen,140,87,81,2


In [5]:
# ---------------------------------------------------- 4. Fungsi pembuatan tabel testing ----------------------------------------------------

#1) Fungsi membuat tabel testing
def create_testing_table_extraction():
    global selected_avg_rgb

    df_testing = pd.DataFrame(columns=["No", "Par", "µr", "µg", "µb", "Hasil"])
    
    image_path_testing = input("Masukkan path ke gambar testing dipstik: ")
    image_test_check = cv2.imread(image_path_testing)
    if image_test_check is None:
        print(f"Error: Gambar testing tidak ditemukan atau tidak dapat dibaca di {image_path_testing}. Ekstraksi dibatalkan.")
        return pd.DataFrame()

    print("\n--- Memulai Ekstraksi Data untuk Tabel Testing ---")
    print("Silahkan memilih piksel berwarna untuk setiap parameter secara berurutan pada gambar testing.")

    for i, param_name in enumerate(PARAM_LIST):
        print(f"\nSilakan pilih warna untuk parameter ke-{i+1}: {param_name} pada gambar testing.")
        
        avg_rgb_values = select_color_from_image(image_path_testing)

        r_val, g_val, b_val = np.nan, np.nan, np.nan # Default jika pemilihan gagal
        if avg_rgb_values is not None:
            r_val, g_val, b_val = avg_rgb_values[0], avg_rgb_values[1], avg_rgb_values[2]
        else:
            print(f"Pemilihan warna untuk {param_name} dibatalkan. Nilai RGB akan kosong.")
            
        new_row_dict = {
            "No": i + 1, # Nomor urut global untuk tabel testing
            "Par": param_name,
            "µr": r_val,
            "µg": g_val,
            "µb": b_val,
            "Hasil": "" # Kolom 'Hasil' akan diisi pada proses klasifikasi di bawah
        }
        df_testing = pd.concat([df_testing, pd.DataFrame([new_row_dict])], ignore_index=True)
        
        clear_output(wait=True) # Membersihkan tampilan di Jupyter

    if not df_testing.empty:
        print("\n--- Tabel Testing Selesai Dibuat ---")
        
        TESTING_TABLE_FILEPATH = input("Silahkan masukkan path file 'Tabel_Testing.csv' untuk disimpan (name.csv): ")
        
        display(df_testing)
        try:
            df_testing.to_csv(TESTING_TABLE_FILEPATH, index=False)
            print(f"Tabel testing berhasil disimpan ke: {TESTING_TABLE_FILEPATH}")
        except Exception as e:
            print(f"Gagal menyimpan tabel testing: {e}")
    else:
        print("Tidak ada data yang dimasukkan ke tabel testing.")
        
create_testing_table_extraction()


--- Tabel Testing Selesai Dibuat ---


Silahkan masukkan path file 'Tabel_Testing.csv' untuk disimpan (name.csv):  TabelTesting.csv


,No,Par,µr,µg,µb,Hasil
0,1,Leukosit,73,59,52,
1,2,Nitrite,107,86,50,
2,3,Urobilnogen,112,79,43,
3,4,Protein,119,111,25,
4,5,pH,86,85,18,
5,6,Blood,156,100,13,
6,7,Specific Gravity,34,52,42,
7,8,Ketone,98,91,62,
8,9,Bilrubin,172,153,79,
9,10,Glucose,92,175,148,


Tabel testing berhasil disimpan ke: TabelTesting.csv


In [9]:
# ---------------------------------------------------- 5. Interpretasi hasil ----------------------------------------------------

# 1. Fungsi Interpretasi
def interpret_hasil(hasil_value):
    
    if isinstance(hasil_value, (int, float)): # Jika hasil adalah angka (float atau int)
        return str(hasil_value)
    elif isinstance(hasil_value, str): # Jika hasil adalah string
        hasil_value_lower = hasil_value.strip().lower() # Bersihkan whitespace dan ubah ke lowercase
        if hasil_value_lower in ['+', '++', '+++', '++++']:
            return "Positif"
        elif hasil_value_lower == '-':
            return "Negatif"
        elif hasil_value_lower == '+/-':
            return "Positif atau Negatif"
        else:
            try:
                return str(float(hasil_value)) # Coba konversi ke angka jika stringnya mirip angka
            except ValueError:
                return hasil_value # Jika tidak dikenali sebagai angka, tampilkan apa adanya
    else:
        return f"Tipe data hasil tidak dikenal: {type(hasil_value).__name__}" # Error handling

try:
    print("------ Interptetasi tabel testing berdasarkan tabel referensi ------\n")

    TabelReferensiInterpretasi = input("Masukkan path file tabel referensi (.csv): ")
    TabelTestingInterpretasi = input("Masukkan path file tabel testing (.csv): ")
    
    df_ref = pd.read_csv(TabelReferensiInterpretasi)
    df_test = pd.read_csv(TabelTestingInterpretasi)

    required_cols = ['Par', 'µr', 'µg', 'µb', 'Hasil']  # Memastikan kolom yang diperlukan ada
    if not all(col in df_test.columns for col in required_cols) or \
       not all(col in df_ref.columns for col in required_cols):
       raise ValueError("File CSV harus mengandung kolom 'Par', 'µr', 'µg', 'µb', dan 'Hasil'.")

except FileNotFoundError as e: # Error handling
    print(f"Error: File tidak ditemukan. Pastikan 'testing.csv' dan 'referensi.csv' ada di direktori yang sama.")
    print(f"Detail Error: {e}")
    exit()
except ValueError as e:
     print(f"Error: Format file CSV tidak sesuai.")
     print(f"Detail Error: {e}")
     exit()
except Exception as e:
    print(f"Terjadi error saat membaca file CSV: {e}")
    exit()

hasil_klasifikasi = [] # Proses Klasifikasi

# Loop setiap baris pada data testing
for index, test_row in df_test.iterrows():
    test_par = test_row['Par']
    test_r = test_row['µr']
    test_g = test_row['µg']
    test_b = test_row['µb']

    print(f"\nMemproses parameter: {test_par}")
    print(f"  RGB Test: ({test_r}, {test_g}, {test_b})")

    # Filter data referensi berdasarkan parameter yang sama
    ref_filtered = df_ref[df_ref['Par'] == test_par].copy()

    if ref_filtered.empty: # Cek apa ada referensi untuk parameter tersebut
        print(f"  Referensi untuk parameter '{test_par}' tidak ditemukan.")
        interpreted_result = f"Referensi tidak ditemukan"
    else:
        ref_filtered['distance'] = ref_filtered.apply( # Hitung jarak Manhattan untuk setiap baris referensi yang difilter
            lambda row: abs(row['µr'] - test_r) + abs(row['µg'] - test_g) + abs(row['µb'] - test_b), # Jarak Manhattan = |r1 - r2| + |g1 - g2| + |b1 - b2|
            axis=1 # Terapkan fungsi per baris
        )

        closest_ref_row = ref_filtered.loc[ref_filtered['distance'].idxmin()] # Cari baris referensi dengan jarak minimum

        # Ambil nilai 'Hasil' dari baris referensi terdekat
        closest_hasil = closest_ref_row['Hasil']
        closest_distance = closest_ref_row['distance']
        closest_rgb_ref = (closest_ref_row['µr'], closest_ref_row['µg'], closest_ref_row['µb'])

        print(f"  RGB Referensi Terdekat: {closest_rgb_ref}")
        print(f"  Jarak Manhattan: {closest_distance}")
        print(f"  Hasil Referensi Terdekat: {closest_hasil}")

        # Interpretasikan hasil
        interpreted_result = interpret_hasil(closest_hasil)
        print(f"  Interpretasi Hasil: {interpreted_result}")

    # Simpan hasil klasifikasi
    hasil_klasifikasi.append({
        'Par': test_par,
        'Interpretasi': interpreted_result
    })

print("\n===== Hasil Klasifikasi Dipstik Urine =====") # Menampilkan hasil akhir
for result in hasil_klasifikasi:
    print(f"{result['Par']} = {result['Interpretasi']}")

------ Interptetasi tabel testing berdasarkan tabel referensi ------



Masukkan path file tabel referensi (.csv):  C:\Users\dhana\Downloads\Tempo\pcd\FinalProjectNew\TabelReferensi.csv
Masukkan path file tabel testing (.csv):  C:\Users\dhana\Downloads\Tempo\pcd\FinalProjectNew\TabelTesting.csv



Memproses parameter: Leukosit
  RGB Test: (73, 59, 52)
  RGB Referensi Terdekat: (57, 34, 42)
  Jarak Manhattan: 51
  Hasil Referensi Terdekat: ++
  Interpretasi Hasil: Positif

Memproses parameter: Nitrite
  RGB Test: (107, 86, 50)
  RGB Referensi Terdekat: (99, 21, 44)
  Jarak Manhattan: 79
  Hasil Referensi Terdekat: +
  Interpretasi Hasil: Positif

Memproses parameter: Urobilnogen
  RGB Test: (112, 79, 43)
  RGB Referensi Terdekat: (122, 72, 71)
  Jarak Manhattan: 45
  Hasil Referensi Terdekat: 4
  Interpretasi Hasil: 4.0

Memproses parameter: Protein
  RGB Test: (119, 111, 25)
  RGB Referensi Terdekat: (117, 102, 51)
  Jarak Manhattan: 37
  Hasil Referensi Terdekat: -
  Interpretasi Hasil: Negatif

Memproses parameter: pH
  RGB Test: (86, 85, 18)
  RGB Referensi Terdekat: (111, 82, 22)
  Jarak Manhattan: 32
  Hasil Referensi Terdekat: 6.0
  Interpretasi Hasil: 6.0

Memproses parameter: Blood
  RGB Test: (156, 100, 13)
  RGB Referensi Terdekat: (113, 86, 22)
  Jarak Manhattan: 66
